***
# Let's try it out!

### Import `kerchunk` and make sure it's at the latest version (`0.0.6` at the time of writing)

In [1]:
import kerchunk
kerchunk.__version__

'0.2.4'

_If Kerchunk is not at the latest version, update with pip/conda: and **restart the kernel**_

In [2]:
# !pip install --upgrade kerchunk

In [3]:
import xarray as xr
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import fsspec
from glob import glob
import json
import earthaccess

## Authenticate to EarthData Login

In [4]:
auth = earthaccess.login(persist=True)

### Find granules according to search parameters, and get S3 URLS

In [5]:
granules = earthaccess.search_data(short_name='M2T1NXSLV', version='5.12.4', temporal=('2024-03-01', '2024-03-31'))

Granules found: 31


In [6]:
urls = [granule.data_links(access='direct', in_region=True)[0] for granule in granules]
urls

['s3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.20240301.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.20240302.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.20240303.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.20240304.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.20240305.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.20240306.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.20240307.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.20240308.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.2024030

Setup an `s3fs` session with authentication

In [7]:
fs = earthaccess.get_s3fs_session(daac='GES_DISC')

### Opening directly with xarray is slow, even in the same region:

In [8]:
%%time
ds = xr.open_mfdataset(
    [fs.open(url) for url in urls], 
)
ds

CPU times: user 44.2 s, sys: 1.91 s, total: 46.1 s
Wall time: 1min 19s


<xarray.Dataset> Size: 29GB
Dimensions:   (lon: 576, lat: 361, time: 744)
Coordinates:
  * lon       (lon) float64 5kB -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * lat       (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * time      (time) datetime64[ns] 6kB 2024-03-01T00:30:00 ... 2024-03-31T23...
Data variables: (12/47)
    CLDPRS    (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    CLDTMP    (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    DISPH     (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    H1000     (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    H250      (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    H500      (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    ...        ...
    V250      (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    V2M       (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    V500      (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    V50M      (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    V850      (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    ZLCL      (time, lat, lon) float32 619MB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
Attributes: (12/30)
    History:                           Original file generated: Wed Mar 13 06...
    Comment:                           GMAO filename: d5124_m2_jan10.tavg1_2d...
    Filename:                          MERRA2_400.tavg1_2d_slv_Nx.20240301.nc4
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    Contact:                           http://gmao.gsfc.nasa.gov
    identifier_product_doi:            10.5067/VJAFPLI1CSIV
    RangeBeginningDate:                2024-03-01
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2024-03-01
    RangeEndingTime:                   23:59:59.000000

## Example of creating a single reference

In [9]:
u = urls[0]
u

's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2024/03/MERRA2_400.tavg1_2d_slv_Nx.20240301.nc4'

Open the file object with `fs.open()`, and process with Kerchunk. The `SingleHdf5ToZarr` class takes in the file object and its url as required arguments. The `inline_threshold` parameter sets the number of bytes a chunk must be to be stored directly in the metadata file (instead of a referenced byte-range). 

`.translate()` returns a `dict` of extracted metadata for use with `ReferenceFileSystem`. This can be aggregated with other metadata or written to disk.

In [10]:
%%time
with fs.open(u) as infile:
    reference = SingleHdf5ToZarr(infile, u).translate()

CPU times: user 9.88 s, sys: 2.1 s, total: 12 s
Wall time: 31.5 s


In [11]:
from IPython.display import JSON

In [12]:
JSON(reference)

<IPython.core.display.JSON object>

## Create references for all files in `flist`

## With a Dask cluster
[Dask](https://dask.org/) is a python package that allows for easily parallelizing python code. This section starts a local client (using whatever processors are available on the current machine). This can also be done just as easily using [Dask clusters](https://docs.dask.org/en/stable/deploying.html). 

In [13]:
import dask
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/lsterzinger/proxy/8787/status,
Dashboard: /user/lsterzinger/proxy/8787/status,Workers: 4
Total threads: 4,Total memory: 14.84 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44447,Workers: 4
Dashboard: /user/lsterzinger/proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 14.84 GiB
Comm: tcp://127.0.0.1:34989,Total threads: 1
Dashboard: /user/lsterzinger/proxy/45325/status,Memory: 3.71 GiB
Nanny: tcp://127.0.0.1:46001,


## Definte function to return a reference dictionary for a given S3 file URL

This function does the following:
1. Use `fs.open()` to open the file given by `url`
2. Using `kerchunk.SingleHdf5ToZarr()` and supplying the file object `infile` and URL `f`, generate reference with `.translate()`

The returned object is a dictionary

In [14]:
def gen_ref(url):
    with fs.open(url) as inf:
        return SingleHdf5ToZarr(inf, url).translate()

### Map `gen_ref` to each member of `flist_bag` and compute
Dask bag is a way to map a function to a set of inputs. This next couple blocks of code tell Dask to take all the files in `flist`, break them up into the same amount of partitions and map each partition to the `gen_ref()` function -- essentially mapping each file path to `gen_ref()`. Calling `bag.compute()` on this runs `gen_ref()` in parallel with as many workers as are available in Dask client.

_Note: if running interactively on Binder, this will take a while since only one worker is available and the references will have to be generated in serial. See option for loading from jsons below_

In [15]:
dicts_delayed = [dask.delayed(gen_ref)(url) for url in urls]
dicts_delayed

[Delayed('gen_ref-1a735416-621d-4462-925c-2c2407ac98c2'),
 Delayed('gen_ref-d3270f37-a193-4224-9302-d329f7566f56'),
 Delayed('gen_ref-2ad7ad21-0ae4-4749-90a7-8f2fa203f818'),
 Delayed('gen_ref-bd8e70a7-b596-42db-8c26-6265fc09ae6e'),
 Delayed('gen_ref-31358370-456b-4957-8d67-f9a9e0ab825b'),
 Delayed('gen_ref-ec3174fd-eb82-46eb-aab9-c3ab1028d3f5'),
 Delayed('gen_ref-54f245c2-2c9e-4875-aecd-c6296b635224'),
 Delayed('gen_ref-742c0116-c42a-4138-bf7b-a27dd911f5e9'),
 Delayed('gen_ref-3e23001e-7ec6-4cbb-b33b-8460ceb3a7e2'),
 Delayed('gen_ref-825d1951-98ad-4895-8089-7446b939c4c5'),
 Delayed('gen_ref-48887b6b-66a1-47fb-b689-0e40dedf3cbc'),
 Delayed('gen_ref-e7b84a25-11e5-4289-9f52-47840f1ca178'),
 Delayed('gen_ref-63861c92-8108-4dfc-891c-f95b665e8682'),
 Delayed('gen_ref-0f848190-1da4-4ba5-8c4a-ecfb9954d8de'),
 Delayed('gen_ref-d3763468-dfe0-44b6-9abe-82295466500a'),
 Delayed('gen_ref-5d2bab9c-cd67-4789-bea8-f5f72ace490a'),
 Delayed('gen_ref-1de9a8b2-1aa9-48b0-8dc1-7cd7e28d429f'),
 Delayed('gen_

In [20]:
%time dicts = dask.compute(*dicts_delayed)

CPU times: user 26.7 s, sys: 4.96 s, total: 31.6 s
Wall time: 6min 4s


Now, each url in `flist` has been used to generate a dictionary of reference data in `dicts`

### _Save/load references to/from JSON files (optional)_
The individual dictionaries can be saved as JSON files if desired

In [17]:
import os
os.makedirs('./jsons', exist_ok=True)

In [21]:
import ast
for d in dicts:
    # Generate name from corresponding URL:
    # Grab URL, strip everything but the filename, 
    # and replace .nc with .json
    json_name = ast.literal_eval(d['refs']['.zattrs'])['Filename'].replace('.nc4', '.json')
        
    with open(f'./jsons/{json_name}', 'w') as outf:
        outf.write(json.dumps(d))

***
### Use `MultiZarrToZarr` to combine the 31 individual references into a single reference
In this example we passed a list of reference dictionaries, but you can also give it a list of `.json` filepaths (commented out)

Need to get S3 access credentials - earthaccess makes this easy.

In [22]:
creds = earthaccess.get_s3_credentials(daac='GES_DISC')

In [24]:
r_opts = {'anon':False,          
          'key':creds['accessKeyId'], 
          'secret':creds['secretAccessKey'], 
          'token':creds['sessionToken']}    #ncfiles

mzz = MultiZarrToZarr(
    # Can be either list of dictionaries or path to jsons
    dicts,
    # './jsons/*.json',

    # Set concat and remote access options
    concat_dims='time',
    remote_options=r_opts,

    # Parse time into cftime before concat.
    # Since values are "seconds since <start of file>"
    # for this dataset, time values are identical for each file. 
    # Not kerchunk compatible without this step
    coo_map={'time' :'cf:time'},
)

References can be saved to a file (`combined.json`) or passed back as a dictionary (`mzz_dict`)

In [25]:
%time mzz.translate('./combined.json')
# mzz_dict = mzz.translate()

CPU times: user 13.8 s, sys: 1.08 s, total: 14.9 s
Wall time: 22.6 s


{'version': 1,
 'refs': {'.zgroup': '{"zarr_format":2}',
  'time/.zarray': '{\n    "chunks": [\n        744\n    ],\n    "compressor": {\n        "id": "zstd",\n        "level": 1\n    },\n    "dtype": "<i8",\n    "fill_value": 4611686018427387904,\n    "filters": null,\n    "order": "C",\n    "shape": [\n        744\n    ],\n    "zarr_format": 2\n}',
  'time/0': 'base64:KLUv/WBAFoUgACZ62DyAbugBICRvSR5E36OwC9nHYfRiNmMp/RiampAoAVCGRQDJgigBUIRFAMmCKCFQhEUAiIIoAFAERQCIKAPHAMkAyACojmX6wA56nUIewllqpJa3ValDuvttDq0PlsMbRNoupyaupzWovNadwf3F9BjDWONn9Mz59YW158JfC6AMJapA2lSkE9WxTBTYQa9TaEI4S40klLdVyUx3vx2h9cFyGIJI2+UHJq6nS5XXuh24v5geNxhr/MzgzPk1ufZc+LsAlKFECkibipQA1bFMIzvodQohCGep0YHytioZSHe/LbY+WA4XEGm7TMDE9fQAlde6Q/cX02MBY42fA5w5vwYBrGAPdIE4+IJgQdhBr1MISDhLjX6Ut1VJTne/7aP1wXK4h0jbZR4mrqdHKq9173B/MT3WMdb4OceZ82vR2nPhLw5QhhLdIG0qkg1UxzLF7KDXKVQjnKVGNMrbqjQj3f023PpgOSxDpO1yDBPX0zBUXusOub+YHr8Ya/zs4sz5dYu158LfF5ShRCxIm4q0AtWxTCrYQa9TyA1nqVGK8rYqoUh3v32i9cFyuBVpu2zCxPV0CZXXukm4v5ges2ONn0ecOb8WsfZc+B